In [17]:
# 传统方式执行代码
import time


def coding():
    for i in range(3):
        print("coding... %s " % i)
        time.sleep(1)


def writing():
    for i in range(3):
        print("writing... %s " % i)
        time.sleep(1)


def main():
    start = time.time()
    coding()
    writing()
    end = time.time()
    print('花费{} s'.format(end - start))


main()

coding... 0 
coding... 1 
coding... 2 
writing... 0 
writing... 1 
writing... 2 
花费6.003387689590454 s


### 多线程实现

In [21]:
import threading

def main():
    t1 = threading.Thread(target=coding)
    t2 = threading.Thread(target=writing)
    
    start = time.time()
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    end = time.time()
    print('花费%s s' %(end-start))
    
main()

coding... 0 
writing... 0 
coding... 1 
writing... 1 
coding... 2 
writing... 2 
花费3.010823965072632 s


In [39]:
class Demo(object):
    @staticmethod
    def coding():
        for i in range(3):
            print("coding... %s " %i)
            time.sleep(1)
    @staticmethod
    def writing():
        for i in range(3):
            print("writing... %s " %i)
            time.sleep(1)
    
    def run(self):
        self.writing()
        self.coding()

demo = Demo()
t1 = threading.Thread(target=demo.run)
t2 = threading.Thread(target=demo.run)

t1.start()
t2.start()

writing... 0 writing... 0 

writing... 1 writing... 1 

writing... 2 writing... 2 

coding... 0 coding... 0 

coding... 1 coding... 1 

coding... 2 
coding... 2 


### 通过类继承实现多线程

In [11]:
import threading

class Coding(threading.Thread):  # 继承threading模块的Thread类
    @staticmethod
    def coding():
        for i in range(3):
            print("coding... %s " %i)
            time.sleep(1)
    
    def run(self):  # 一定要重写run() 方法，然后在创建实例的时候就会自动创建一个线程
        self.coding()
        
        
class Writing(threading.Thread):
    @staticmethod
    def writing():
        for i in range(3):
            print("writing... %s " %i)
            time.sleep(1)
    
    def run(self):
        self.writing()

def main():
    t1 = Coding()  # 创建线程
    t2 = Writing()  # 创建线程
    print(threading.enumerate())
    
    t1.start()
    t2.start()

main()

[<_MainThread(MainThread, started 13100)>, <Thread(Thread-4, started daemon 12148)>, <Heartbeat(Thread-5, started daemon 7900)>, <HistorySavingThread(IPythonHistorySavingThread, started 13524)>, <ParentPollerWindows(Thread-3, started daemon 1992)>, <Consumer(消费者:1, started 13548)>, <Producer(生产者:1, started 8736)>, <Consumer(消费者:0, started 5396)>, <Consumer(消费者:1, started 3768)>, <Consumer(消费者:2, started 17088)>, <Producer(生产者:1, started 16296)>, <Producer(生产者:2, started 13388)>, <Producer(生产者:3, started 1600)>, <Producer(生产者:4, started 4476)>, <Consumer(消费者:1, started 12064)>, <Consumer(消费者:2, started 15108)>]
coding... 0 
writing... 0 
coding... 1 
writing... 1 
coding... 2 
writing... 2 


### 多线程共享全局变量


In [86]:
# 多线程共享全局变量有可能会带来意想不到的错误

VALUE = 0


def add():
    global VALUE
    for i in range(100000):
        VALUE += 1
    print("value is %d" % VALUE)


def main():
    for i in range(2):
        t = threading.Thread(target=add)
        t.start()


main()

value is 145958
value is 152254


In [87]:
# 解决方案：上锁
# 上锁的地方要在 那些修改全局变量的地方

VALUE = 0
g_lock = threading.Lock()  # 实例化一个锁


def add():
    global VALUE
    g_lock.acquire()  # 上锁
    for i in range(100000):
        #         g_lock.acquire()
        VALUE += 1
    g_lock.release()  # 解锁
    print("value is %d" % VALUE)


def main():
    for i in range(2):
        t = threading.Thread(target=add)
        t.start()


main()

value is 100000
value is 200000


### Lock版生产者与消费者模式

In [7]:
import threading
import random
import time

# G_MONEY：金库里面的钱
G_MONEY = 1000
# 创建一个全局锁
g_lock = threading.Lock()
G_TOTAL_TIME = 10
g_time = 0


class Producer(threading.Thread):
    def run(self):
        global G_MONEY
        global g_time
        while True:
            money = random.randint(100, 1000)
            g_lock.acquire()
            if g_time > G_TOTAL_TIME:
                g_lock.release()
                break
            G_MONEY += money
            print("{}：生成了{} 元，还剩{} 元".format(threading.current_thread(), money,
                                             G_MONEY))
            g_time += 1
            g_lock.release()
            time.sleep(1)


class Consumer(threading.Thread):
    def run(self):
        global G_MONEY
#         global g_time
        while True:
            money = random.randint(100, 1000)
            g_lock.acquire()
            if g_time >= G_TOTAL_TIME:
                    g_lock.release()
                    break
                    
            elif G_MONEY >= money:
                G_MONEY -= money
                print("{}:消费了{} 元，还剩{} 元".format(threading.current_thread(),
                                                 money, G_MONEY))
            else:
                print("{}准备消费{} 元，金额不足。。。剩余{}元。。。。。。。。。".format(
                    threading.current_thread(), money, G_MONEY))
            g_lock.release()
            time.sleep(1)


def main():
    # 实例化3个消费者线程
    for i in range(3):
        t = Consumer(name="消费者:{}".format(i))
        t.start()

    # 实例化5个生产者线程
    for i in range(5):
        t = Producer(name='生产者:{}'.format(i))
        t.start()


if __name__ == "__main__":
    main()

<Consumer(消费者:0, started 2464)>:消费了721 元，还剩279 元
<Consumer(消费者:1, started 4956)>准备消费652 元，金额不足。。。剩余279元。。。。。。。。。
<Consumer(消费者:2, started 4456)>准备消费775 元，金额不足。。。剩余279元。。。。。。。。。
<Producer(生产者:0, started 7256)>：生成了346 元，还剩625 元
<Producer(生产者:1, started 15280)>：生成了183 元，还剩808 元
<Producer(生产者:2, started 8048)>：生成了587 元，还剩1395 元
<Producer(生产者:3, started 5812)>：生成了501 元，还剩1896 元
<Producer(生产者:4, started 14656)>：生成了529 元，还剩2425 元
<Consumer(消费者:0, started 2464)>:消费了520 元，还剩1905 元
<Consumer(消费者:1, started 4956)>:消费了673 元，还剩1232 元
<Consumer(消费者:2, started 4456)>:消费了221 元，还剩1011 元
<Producer(生产者:1, started 15280)>：生成了855 元，还剩1866 元
<Producer(生产者:0, started 7256)>：生成了653 元，还剩2519 元
<Producer(生产者:3, started 5812)>：生成了337 元，还剩2856 元
<Producer(生产者:2, started 8048)>：生成了928 元，还剩3784 元
<Producer(生产者:4, started 14656)>：生成了603 元，还剩4387 元
<Producer(生产者:0, started 7256)>：生成了825 元，还剩5212 元


In [3]:
# 重复上锁解锁浪费资源

### condition版消费者和生产者

In [13]:
import threading
import random
import time

# G_MONEY：金库里面的钱
G_MONEY = 1000
# 创建一个全局锁
g_condition_lock = threading.Condition()
G_TOTAL_TIME = 10
g_time = 0


class Producer(threading.Thread):
    def run(self):
        global G_MONEY
        global g_time
        while True:
            money = random.randint(100, 1000)
            g_condition_lock.acquire()
            if g_time > G_TOTAL_TIME:
                g_condition_lock.release()
                break
            G_MONEY += money
            print("{}：生成了{} 元，还剩{} 元".format(threading.current_thread(), money,
                                             G_MONEY))
            g_time += 1
            g_condition_lock.notify_all()  # 同时偷偷换了锁的同志把锁换回来
            g_condition_lock.release()
            time.sleep(1)


class Consumer(threading.Thread):
    def run(self):
        global G_MONEY
        global g_time
        while True:
            money = random.randint(900, 1000)
            g_condition_lock.acquire()
            while G_MONEY <= money:
                if g_time >= G_TOTAL_TIME:
                    g_condition_lock.release()
                    return 
                print("d等待中....")
                g_condition_lock.wait()  # 解开g_condition_lock 上了一把特殊的锁，如果有其他线程通知的话，会换回原来的锁
            G_MONEY -= money
            print("{}消费了{}元， 剩余{} 元".format(threading.current_thread(), money, G_MONEY))
            g_condition_lock.release()
            time.sleep(1)


def main():
    # 实例化3个消费者线程
    for i in range(3):
        t = Consumer(name="消费者:{}".format(i))
        t.start()

    # 实例化5个生产者线程
    for i in range(5):
        t = Producer(name='生产者:{}'.format(i))
        t.start()


if __name__ == "__main__":
    main()

<Consumer(消费者:0, started 12144)>消费了951元， 剩余49 元
d等待中....
d等待中....
<Producer(生产者:0, started 9624)>：生成了258 元，还剩307 元
d等待中....
d等待中....
<Producer(生产者:1, started 8320)>：生成了810 元，还剩1117 元
<Consumer(消费者:2, started 7688)>消费了921元， 剩余196 元
d等待中....
<Producer(生产者:2, started 16312)>：生成了554 元，还剩750 元
d等待中....
<Producer(生产者:3, started 16600)>：生成了634 元，还剩1384 元
<Producer(生产者:4, started 1892)>：生成了897 元，还剩2281 元
<Consumer(消费者:1, started 10804)>消费了993元， 剩余1288 元
<Producer(生产者:0, started 11592)>：生成了359 元，还剩1647 元
<Consumer(消费者:0, started 2224)>消费了948元， 剩余699 元
<Producer(生产者:1, started 14896)>：生成了349 元，还剩1048 元
<Consumer(消费者:1, started 12460)>消费了945元， 剩余103 元
<Producer(生产者:2, started 1288)>：生成了716 元，还剩819 元
<Producer(生产者:3, started 1420)>：生成了331 元，还剩1150 元
<Producer(生产者:4, started 8472)>：生成了272 元，还剩1422 元
<Consumer(消费者:2, started 10028)>消费了971元， 剩余451 元
<Producer(生产者:0, started 9624)>：生成了649 元，还剩1100 元
<Consumer(消费者:2, started 7688)>消费了979元， 剩余121 元


### 队列
+ 先进先出
+ 要做就做完

In [2]:
from queue import Queue

q = Queue(4)  # 实例化一个队列，并指定队列的大小
q.put(1)  # 往队列里面添加数据
q.put(2)
for i in range(3,5):
    q.put(i)

print(q.qsize())  # q.qsize() 返回队列当前的大小

print(q.full())  # 判断队列是否满了，如果满了返回True，不满返回False
print(q.empty())  # 如果队列为空则返回True, 不为空则返回False

for i in range(4):
    print(q.get())  # 从队列中取出数据，按照先进先出原则

4
True
False
1
2
3
4


`q.get(block=True)`  # 当队列为空时，如果还往队列里面取值，则会堵塞(默认堵塞)  
`q.put(block=True)`  # 当队列满时，如果还往队列加数据，则会堵塞(默认堵塞)

In [2]:
from threading import Thread
from queue import Queue
import time
index = 0


def put_value(q):
    global index
    while index <= 10:
        q.put(index)
        index += 1
        print("准备睡觉")
        time.sleep(1)
        

def get_value(q):
    global index
    while index<=10:
        print(q.get())


def main():
    q = Queue(4)
    t1 = Thread(target=put_value, args=(q, ))
    t2 = Thread(target=get_value, args=(q, ))

    t1.start()
    t2.start()


if __name__ == "__main__":
    main()

准备睡觉
0
准备睡觉1

准备睡觉2

准备睡觉3

准备睡觉4

准备睡觉5

准备睡觉6

准备睡觉7

准备睡觉8

准备睡觉9

准备睡觉10

